In [1]:
import pandas as pd

In [77]:
d = {'LocMention': ['A','A','A','A','A','A', 'B','B','B','B','C','C','C','C'], \
     'LocEvent': ['A','A','A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D'], \
     'EventRootCode':['01','02', '01','02', '01','02','01','02','01','02','01','02','01','02'], \
     'NumberofInterMentions':[1,2,3,4,5,6,7,8, 9, 10, 11, 12,13, 14]}
df = pd.DataFrame(data=d)

In [78]:
df.head()

,LocMention,LocEvent,EventRootCode,NumberofInterMentions
0,A,A,01,1
1,A,A,02,2
2,A,A,01,3
3,A,B,02,4
4,A,C,01,5


In [ ]:
#test = df.groupby(['LocMention']).agg({'NumberofInterMentions': 'sum'})
#test_new = test.groupby('country').filter(lambda df: any(df.EventRootCode == '19'))

In [79]:
test = df.groupby(['LocMention','LocEvent']).agg({'NumberofInterMentions': 'sum'})
test.head()

NumberofInterMentions
LocMention LocEvent                       
A          A                             6
           B                             4
           C                             5
           D                             6
B          A                             7

In [80]:
test_new = test.groupby(level=['LocMention']).apply(lambda x: 100 * x / float(x.sum()))
test_new.head()

NumberofInterMentions
LocMention LocEvent                       
A          A                     28.571429
           B                     19.047619
           C                     23.809524
           D                     28.571429
B          A                     20.588235

In [68]:
test_new.rename(index=str, columns={"NumberofInterMentions": "Ratio"}, inplace=True)

NumberofInterMentions
LocMention LocEvent                       
A          A                             1
           B                             2
           C                             3
           D                             4
B          A                             5

In [69]:
test_new.reset_index()

,LocMention,LocEvent,Ratio
0,A,A,10.000000
1,A,B,20.000000
2,A,C,30.000000
3,A,D,40.000000
4,B,A,19.230769
5,B,B,23.076923
6,B,C,26.923077
7,B,D,30.769231
8,C,A,21.428571
9,C,B,23.809524


In [72]:
test_new

,LocMention,LocEvent,Ratio,EventRootCode,NumberofInterMentions
0,A,A,10.000000,01,1
1,A,B,20.000000,02,2
2,A,C,30.000000,01,3
3,A,D,40.000000,02,4
4,B,A,19.230769,01,5
5,B,B,23.076923,02,6
6,B,C,26.923077,01,7
7,B,D,30.769231,02,8
8,C,A,21.428571,01,9
9,C,B,23.809524,02,10


In [70]:
test_new = test_new.merge(df, left_on=['LocMention', 'LocEvent'], right_on=['LocMention', 'LocEvent'])

In [71]:
test_new

,LocMention,LocEvent,Ratio,EventRootCode,NumberofInterMentions
0,A,A,10.000000,01,1
1,A,B,20.000000,02,2
2,A,C,30.000000,01,3
3,A,D,40.000000,02,4
4,B,A,19.230769,01,5
5,B,B,23.076923,02,6
6,B,C,26.923077,01,7
7,B,D,30.769231,02,8
8,C,A,21.428571,01,9
9,C,B,23.809524,02,10


In [26]:
# Initial grouping (basically a sorted version of df)
PreGroupby_df = df.groupby(["col1","col2"]).agg({'col3': 'sum'}).reset_index()
PreGroupby_df

,col1,col2,col3
0,A,A,1
1,A,B,2
2,A,C,3
3,A,D,4
4,B,A,5
5,B,B,6
6,B,C,7
7,B,D,8
8,C,A,9
9,C,B,10


In [27]:
# Get the sum of values for the "final group", append "_Sum" to it's column name, and change it into a dataframe (.reset_index)
SumGroup_df = df.groupby(["col1"]).agg({'col3': 'sum'}).add_suffix('_Sum').reset_index()
SumGroup_df

,col1,col3_Sum
0,A,10
1,B,26
2,C,42


In [28]:
# Merge the two dataframes
Percents_df = pd.merge(PreGroupby_df, SumGroup_df)
Percents_df

,col1,col2,col3,col3_Sum
0,A,A,1,10
1,A,B,2,10
2,A,C,3,10
3,A,D,4,10
4,B,A,5,26
5,B,B,6,26
6,B,C,7,26
7,B,D,8,26
8,C,A,9,42
9,C,B,10,42


In [29]:
# Divide the two columns
Percents_df["Percent of Final Group"] = Percents_df["col3"] / Percents_df["col3_Sum"] * 100
Percents_df

,col1,col2,col3,col3_Sum,Percent of Final Group
0,A,A,1,10,10.000000
1,A,B,2,10,20.000000
2,A,C,3,10,30.000000
3,A,D,4,10,40.000000
4,B,A,5,26,19.230769
5,B,B,6,26,23.076923
6,B,C,7,26,26.923077
7,B,D,8,26,30.769231
8,C,A,9,42,21.428571
9,C,B,10,42,23.809524


In [ ]:
# Drop the extra _Sum column
Percents_df.drop(["col3_Sum"], inplace=True, axis=1)